In [55]:
from dart_fss import get_corp_list
from dotenv import load_dotenv
import dart_fss as dart
import pandas as pd
import requests
import json
import os
import time

def get_keys_by_value(d, target_value): # 딕셔너리에서 value로 key값 출력하는 함수
    return [k for k, v in d.items() if v == target_value]

load_dotenv()

API_KEY = os.getenv("API_KEY")

dart.set_api_key(api_key=API_KEY)

# 모든 상장된 기업 리스트 불러오기
corp_list = get_corp_list()


# 삼성전자를 이름으로 찾기 ( 리스트 반환 )
samsung = corp_list.find_by_corp_name('삼성전자', exactly=True)[0]
samsung

Label,Data
corp_code,00126380
corp_name,삼성전자
corp_eng_name,"SAMSUNG ELECTRONICS CO,.LTD"
stock_code,005930
modify_date,20250326
sector,통신 및 방송 장비 제조업
product,"통신 및 방송 장비 제조(무선) 제품, 반도체 제조(메모리) 제품, 전자부품 제조(디스플레이) 제품, 영상 및 음향기기 제조(영상기기) 제품 등"
corp_cls,Y


In [64]:
corp_list.find_by_corp_name('삼성전자')
corp_name_list = ["삼성전자", "LG전자", "에코프로", "현대중공업", "현대자동차"]
target_accounts = ["유동자산", "비유동자산", "자산총계", "부채총계"]

corp_name_dict = {}

for corp in corp_name_list:
    name = corp_list.find_by_corp_name(corp, exactly=True)
    if name:
        corp_code = str(name[0])[1:9]
    else:
        print(f"{corp}(은)는 다중회사가 아니므로 건너뜁니다.")
        continue
    corp_name_dict[f"{corp}"] = corp_code
    print(f"{corp}의 회사 이름과 고유번호를 딕셔너리에 추가했습니다.")

print(f"회사명&고유번호 딕셔너리: {corp_name_dict}")

# 저장할 데이터를 리스트 형태로 정리
rows = []

삼성전자의 회사 이름과 고유번호를 딕셔너리에 추가했습니다.
LG전자의 회사 이름과 고유번호를 딕셔너리에 추가했습니다.
에코프로의 회사 이름과 고유번호를 딕셔너리에 추가했습니다.
현대중공업(은)는 다중회사가 아니므로 건너뜁니다.
현대자동차의 회사 이름과 고유번호를 딕셔너리에 추가했습니다.
회사명&고유번호 딕셔너리: {'삼성전자': '00126380', 'LG전자': '00401731', '에코프로': '00536541', '현대자동차': '00164742'}


In [66]:
# --- API 요청 URL 및 파라미터 ---
for corp_name, corp_code in corp_name_dict.items():
    base_url = "https://opendart.fss.or.kr/api/fnlttMultiAcnt.json"
    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code, # 대상 기업 코드
        'bsns_year': "2024", # 회계연도
        'reprt_code': "11011", # 사업보고서
    }

    # --- GET 요청 보내기 ---
    try:
        response = requests.get(base_url, params=params)
        
        # --- 응답 확인 ---
        # HTTP 상태 코드가 200 (OK)인지 확인
        if response.status_code == 200:
            print("요청 성공!")

            # Requests 형식을 json으로 변환
            data = json.loads(response.text)

            for item in data.get("list", []):
                account_nm = item.get("account_nm")
                if account_nm in target_accounts:
                    row = {
                        "기업명": corp_name,
                        "기업고유번호": corp_code,
                        "계정명": account_nm,
                        "당기명": item.get("thstrm_nm"),
                        "당기일자": item.get("thstrm_dt"),
                        "당기금액": item.get("thstrm_amount"),
                        "전기명": item.get("frmtrm_nm"),
                        "전기일자": item.get("frmtrm_dt"),
                        "전기금액": item.get("frmtrm_amount"),
                        "전전기명": item.get("bfefrmtrm_nm"),
                        "전전기일자": item.get("bfefrmtrm_dt"),
                        "전전기금액": item.get("bfefrmtrm_amount"),
                        "공시번호": item.get("rcept_no"),
                        "재무제표종류": item.get("fs_nm"),
                        "재무제표명": item.get("sj_nm")
                    }
                    rows.append(row)
            print(f"{corp_name}의 주요계정을 DataFrame에 추가했습니다.")
        else:
            print(f"에러 발생: HTTP 상태 코드 {response.status_code}")

        time.sleep(0.3)

    except requests.exceptions.RequestException as e:
        print(f"요청 중 에러 발생: {e}")

요청 성공!
삼성전자의 주요계정을 DataFrame에 추가했습니다.
요청 성공!
LG전자의 주요계정을 DataFrame에 추가했습니다.
요청 성공!
에코프로의 주요계정을 DataFrame에 추가했습니다.
요청 성공!
현대자동차의 주요계정을 DataFrame에 추가했습니다.


In [71]:
# DataFrame 생성
df = pd.DataFrame(rows)

# CSV 저장
# df.to_csv("계정별_재무데이터.csv", index=False, encoding='utf-8-sig')

In [69]:

# --- API 요청 URL 및 파라미터 ---
base_url_1 = "https://opendart.fss.or.kr/api/fnlttCmpnyIndx.json"
params = {
    'crtfc_key': API_KEY,
    'corp_code': corp_code, # 대상 기업 코드
    'bsns_year': "2024", # 회계연도
    'reprt_code': "11011", # 사업보고서
    'idx_cl_code': "M220000" # 수익성지표 : M210000 안정성지표 : M220000 성장성지표 : M230000 활동성지표 : M240000
}

# --- GET 요청 보내기 ---
try:
    response_1 = requests.get(base_url_1, params=params)
    
    # --- 응답 확인 ---
    # HTTP 상태 코드가 200 (OK)인지 확인
    if response_1.status_code == 200:
        # XML 형식의 응답 내용을 출력
        print("요청 성공!")
        # print(response_1.text) 분석&코드 작성 시 사용
    else:
        print(f"에러 발생: HTTP 상태 코드 {response_1.status_code}")

except requests.exceptions.RequestException as e:
    print(f"요청 중 에러 발생: {e}")

요청 성공!


In [70]:
data = json.loads(response_1.text)

# 저장할 데이터를 리스트 형태로 정리
rows = []

target_ratio = ["자기자본비율", "부채비율", "유동비율", "당좌비율", "유동부채비율", "이자보상배율"]

for item in data.get("list", []):
    idx_nm = item.get("idx_nm")
    if idx_nm in target_ratio:
        row = {
            "사업연도": item.get("bsns_year"),
            "결산기준일": item.get("stlm_dt"),
            "지표명": item.get("idx_nm"),
            "지표값": item.get("idx_val"),
        }
        rows.append(row)

# DataFrame 생성
df = pd.DataFrame(rows)
df

# CSV 저장
# df.to_csv("주요지표.csv", index=False, encoding='utf-8-sig')

,사업연도,결산기준일,지표명,지표값
0,2024,2024-12-31,자기자본비율,35.396
1,2024,2024-12-31,부채비율,182.516
2,2024,2024-12-31,유동비율,145.598
3,2024,2024-12-31,당좌비율,None
4,2024,2024-12-31,유동부채비율,66.106
5,2024,2024-12-31,이자보상배율,None
